In [1]:
import glob

import chardet
import numpy as np
import pandas as pd
# import zipfile

from cleannumbers import CleanNumbers as cn
from cleantext import CleanText as ct

file_list = glob.glob('../../COVID-19-ALL/csse_covid_19_data/csse_covid_19_daily_reports/*.csv', recursive=True)

df = pd.DataFrame()
for f in file_list:
    rawdata = open(f, 'rb').read()
    result = chardet.detect(rawdata)
    charenc = result['encoding']

    df_temp = pd.read_csv(f, engine='python', delimiter=',', quotechar='"', encoding=charenc, infer_datetime_format=True)
    
        # if "Last Update" in df_temp.columns:
    df_temp.rename(columns={'Last Update': 'Last_Update', 'Latitude': 'Lat', 'Longitude': 'Long_', 'Province/State': 'Province_State', 'Country/Region': 'Country_Region'}, inplace=True)
    cols = df_temp.columns
    # if len(cols) == 1:
    #     print(df_temp.head())
    if ("Lat" or "Long_") not in cols:
        df_temp['Lat'] = np.nan
        df_temp['Long_'] = np.nan
    if "Combined_Key" not in cols:
        df_temp['Combined_Key'] = np.nan
    
    df_temp = df_temp[df_temp['Country_Region']=='US']

    df = df.append(df_temp, ignore_index=True)

df.head()

,Province_State,Country_Region,Last_Update,Confirmed,Deaths,Recovered,Lat,Long_,Combined_Key,FIPS,Admin2,Active
0,"Ashland, NE",US,2020-02-21T23:23:06,11.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,"Travis, CA",US,2020-02-21T23:23:06,5.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,"Chicago, IL",US,2020-02-09T19:03:03,2.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
3,"Lackland, TX",US,2020-02-21T23:23:06,2.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,"San Benito, CA",US,2020-02-03T03:53:02,2.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
df['Last_Update'] = pd.to_datetime(df['Last_Update'], utc=True)
df['Last_Update'] = df['Last_Update'].dt.date
# a = cn(df)
# x = a.clean_numbers(verbose=True, rpl_empty_with='zeros', drop_na=False, exceptions=['Lat', 'Long_', 'FIPS'])
# ints = ["Confirmed", "Deaths", "Recovered", "FIPS"]
for c in  ints:
    df[c].fillna(0, inplace=True)
    df[c] = df[c].astype(np.int64)

df= df.drop_duplicates()

# start_date = pd.to_datetime(['2020-03-02']).date
# end_date = pd.to_datetime(['2020-03-02']).date
# mask = (df['Last_Update'] >= start_date[0]) & (df['Last_Update'] <= end_date[0])
# df1 = df.loc[mask]
# print(df1.Confirmed.sum())
# df1

In [56]:
save_path = os.getcwd()
save_file = os.path.join(save_path, 'covid-us-aggd.csv')

df.to_csv(save_file, index=False)